# Analyse des resultats des test de différentes structures de Rsnet sur le dataset Cifar-10

### Informations 

Dans ce notebook, nous allons analyser les resultats obtenus sur la première vague de tests de différentes structures et hyperparametres de Rsnet. Pour les tests menés, pour chaque nombre de couches cachées de 3 à 9, le modèle est testé sans regularisation et sans dropout, avec deux filtre à chaque fois (32 ou 64), puis avec regularisation, avec dropout et enfin dropout et regularisation sur le modèle, en plus de quelques modèles testés avec du Maxpoolin. En vue de la grande quantité d'architectures testées, nous allons mettre suelement un ou deux exemples do nos méthodes d'analyse des résultats à titre explicatif, et présenter les résultats définitifs.

## Preparation des outils

In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
from PIL import Image

# --------------------------------------------------------------------------------------------------------------

# Rsnet Sans régularisation

### Tests réalisés :
    Pour ce début de tests, nous avons essayé d'entrainer différents Rsnet en jouant sur le nombre de couches ainsi que leur filtres. Pour cela, on a lancé des tests comme suit (plusieurs fois sur la même structure) :
    - Des modèles de 3 à 4 couches, avec filtres entre 32 et 64.
    - Des modèles de 5 à 6 couches, avec filtres entre 32 et 64.
    - Des modèles de 7 à 8 couches, avec filtres entre 32 et 64.
 
    
    Nous avons ensuite enregistré les résultats obtenus dans des fichiers CSV, en plus des logs générés par tensorflow, pour étudier les résultats obtenus.
    
    Nous allons donc charger ces statistiques et voir ce que ce genre de modèles peut donner sur le Dataset 

## Image illustrant un resnet de 5 couches avec un skip connexion de 2:

![Rsnet Model](Rsnet.png)

## Chargement des résultats obtenus pour chaque configuration de modèle:

In [80]:
results_dataframes_list = []
results_dataframes_desc = []
root_csv_folder = "..\\historique_tests\\"


# Rsnet without regularisation results  3 to 4 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_3_to_4_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_3_to_4_{}".format(i))

# Rsnet without regularisation results  5 to 6 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_5_to_6_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_5_to_6_{}".format(i))

# Rsnet without regularisation results  7 to 8 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_7_to_8_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_7_to_8_{}".format(i))



### Extraction des informations interessantes :
- Pire résultat sur le train
- Pire résultat sur la val
- Meilleur résultat sur le train
- Meilleur résultat sur la val 
- Moyenne des résultats sur le train 
- Moyenne des résultats sur la val 
- Mediane des résultat sur le train 
- Mediane des résultat sur la val

In [81]:
def print_statistics(history_df, history_df_desc):
    # Calculating statistics on all models
    if(not history_df.empty):
        sorted_train_accuracy = np.sort(history_df.train_accuracy)
        sorted_val_accuracy = np.sort(history_df.val_accuracy)

        max_val_accuracy = sorted_val_accuracy[-1]
#         second_max_val_accuracy = sorted_val_accuracy[-2]
        min_val_accuracy = history_df.val_accuracy.min()
        mean_val_accuracy = history_df.val_accuracy.mean()
        median_val_accuracy = history_df.val_accuracy.median()

        max_train_accuracy = sorted_train_accuracy[-1]
#         second_max_train_accuracy = sorted_train_accuracy[-2]
        min_train_accuracy = history_df.train_accuracy.min()
        mean_train_accuracy = history_df.train_accuracy.mean()
        median_train_accuracy = history_df.train_accuracy.median()

        # Printing models statistics
        print()
        print("########### Models {} ###########".format(history_df_desc))
        print(history_df.shape)
        print("Pire résultat sur le train : ", min_train_accuracy)
        print("Pire Résultat sur la val : ", min_val_accuracy)

        print("Meilleur résultat sur le train : ", max_train_accuracy)
        print("Meilleur résultat sur la val : ", max_val_accuracy)

        print("Moyenne des résultats sur le train : ", mean_train_accuracy)
        print("Moyenne des résultats sur la val : ", mean_val_accuracy)

        print("Mediane des résultat sur le train : ", median_train_accuracy)
        print("Mediane des résultat sur la val : ", median_val_accuracy)

def filter_on_mean_val_accuracy(results_dataframes_list):
    filtered_results_dataframes = []
    
    for i in range(len(results_dataframes_list)):

        # Calculating statistics on all models
        history_df = results_dataframes_list[i]
        history_df_desc = results_dataframes_desc[i]

        median_val_accuracy = history_df.val_accuracy.median()
        mean_val_accuracy = history_df.val_accuracy.mean()
        # Filtering models to get the best ones (those whose val accuracy is higher then the median)
        
        # Si la médiane est inférieure à la moyenne on enlève cette archi, car la majorité des modèles on une mauvaise
        # accuracy, et la moyenne est gonflée par seulement une petite partie des modèles. Ce qui indique que que l'archi
        # choisie n'est pas interessante
#         if(mean_val_accuracy < median_val_accuracy):
        filtered_history = history_df[history_df.val_accuracy >= mean_val_accuracy]
#         else:
#         filtered_history = pd.DataFrame(columns=history_df.columns)
        filtered_results_dataframes.append(filtered_history)
        
    
    return filtered_results_dataframes


def filter_on_mean_train_accuracy(results_dataframes_list):
    filtered_results_dataframes = []
    
    for i in range(len(results_dataframes_list)):

        # Calculating statistics on all models
        history_df = results_dataframes_list[i]
        history_df_desc = results_dataframes_desc[i]

#         median_train_accuracy = history_df.train_accuracy.median()
        mean_train_accuracy = history_df.train_accuracy.mean()
        # Filtering models to get the best ones (those whose val accuracy is higher then the median)
        
        # Si la médiane est inférieure à la moyenne on enlève cette archi, car la majorité des modèles on une mauvaise
        # accuracy, et la moyenne est gonflée par seulement une petite partie des modèles. Ce qui indique que que l'archi
        # choisie n'est pas interessante
        filtered_history = history_df[history_df.train_accuracy > mean_train_accuracy]
        filtered_results_dataframes.append(filtered_history)
        
    
    return filtered_results_dataframes

def concat_list_of_dataframes(list_df):
    return pd.concat(list_df)

In [82]:
for i in range(len(results_dataframes_list)):
    history = results_dataframes_list[i]
    history_desc = results_dataframes_desc[i]
    print_statistics(history, history_desc)


########### Models rsnet_3_to_4_32 ###########
(6, 26)
Pire résultat sur le train :  0.79602
Pire Résultat sur la val :  0.4105
Meilleur résultat sur le train :  0.9412
Meilleur résultat sur la val :  0.5181
Moyenne des résultats sur le train :  0.8457
Moyenne des résultats sur la val :  0.4708666666666667
Mediane des résultat sur le train :  0.8299300000000001
Mediane des résultat sur la val :  0.4709

########### Models rsnet_3_to_4_64 ###########
(6, 26)
Pire résultat sur le train :  0.67818
Pire Résultat sur la val :  0.3798
Meilleur résultat sur le train :  0.8758799999999999
Meilleur résultat sur la val :  0.5403
Moyenne des résultats sur le train :  0.8202166666666667
Moyenne des résultats sur la val :  0.4602833333333334
Mediane des résultat sur le train :  0.84825
Mediane des résultat sur la val :  0.46085

########### Models rsnet_5_to_6_32 ###########
(8, 26)
Pire résultat sur le train :  0.2551
Pire Résultat sur la val :  0.2447
Meilleur résultat sur le train :  0.97037999

###### On ne garde que les modèles dont les résultats sur la validations sont supérieurs à la moyenne des résultats de tous les modèles (d'une même architecture) testés 

In [83]:
filtered_results_dataframes = filter_on_mean_val_accuracy(results_dataframes_list)

In [84]:
for i in range(len(filtered_results_dataframes)):
    filtered_history = filtered_results_dataframes[i]
    filtered_history_desc = results_dataframes_desc[i]
    print_statistics(filtered_history, filtered_history_desc)


########### Models rsnet_3_to_4_32 ###########
(3, 26)
Pire résultat sur le train :  0.80794
Pire Résultat sur la val :  0.48
Meilleur résultat sur le train :  0.8313200000000001
Meilleur résultat sur la val :  0.5181
Moyenne des résultats sur le train :  0.8226
Moyenne des résultats sur la val :  0.5000666666666667
Mediane des résultat sur le train :  0.8285399999999999
Mediane des résultat sur la val :  0.5021

########### Models rsnet_3_to_4_64 ###########
(3, 26)
Pire résultat sur le train :  0.79588
Pire Résultat sur la val :  0.4929
Meilleur résultat sur le train :  0.8758799999999999
Meilleur résultat sur la val :  0.5403
Moyenne des résultats sur le train :  0.8442866666666666
Moyenne des résultats sur la val :  0.5111
Mediane des résultat sur le train :  0.8611
Mediane des résultat sur la val :  0.5001

########### Models rsnet_5_to_6_32 ###########
(4, 26)
Pire résultat sur le train :  0.41758
Pire Résultat sur la val :  0.4159
Meilleur résultat sur le train :  0.97037999999

###### On fusionne tous les dataframes en une seule (étant donné qu'il y en a une par structure de modèle)

In [85]:
merged_df = concat_list_of_dataframes(filtered_results_dataframes)

In [86]:
print_statistics(merged_df, "Rsnet_without_regularisation")


########### Models Rsnet_without_regularisation ###########
(21, 26)
Pire résultat sur le train :  0.41758
Pire Résultat sur la val :  0.4159
Meilleur résultat sur le train :  0.9838600000000001
Meilleur résultat sur la val :  0.6023
Moyenne des résultats sur le train :  0.8794609523809525
Moyenne des résultats sur la val :  0.5361809523809524
Mediane des résultat sur le train :  0.9395
Mediane des résultat sur la val :  0.5398


### Première analyse des résultats obtenus :

#### Nous remarquons donc sur les 19 meilleurs modèles, nous tournons autours des 60% de résultat sur la validation, et 97% sur le train. De plus, nous avons atteint une limite de taille à lancer, étant donné que nos fits crashent pour manque de mémoire GPU quand on essaye de s'attaquer à des modèles avec trop couches cachées et un batch size importatnt. 

#### Ces résultats nous montrent que les meilleurs modèles overfittent. Nous allons donc essayer d'appliquer des techniques de régularisation dessus, en prenant les meilleurs sur les modèles selectionnés. Mais avant cela, nous allons essayer de réduire le nombre de modèles à tester on ne prenant que les plus intéressants.

## Image representant un overfiting sur un Rsnet :

![Rsnet_overfiting](Overfit_Rsnet.PNG)

## Selection des meilleurs modèles sur la val sur les modèles selectionnés:
    Pour cela, nous n'allons prendre que les modèles dont les résultats sur l'entraînement sont supérieurs à la médiane des résultats de tous les modèles compris (toutes arachitectures confondues).

In [87]:
best_val_models = filter_on_mean_val_accuracy([merged_df])[0]
best_val_models = best_val_models.sort_values('val_accuracy', ascending=False)
print_statistics(best_val_models, "Best Rsnet models")


########### Models Best Rsnet models ###########
(12, 26)
Pire résultat sur le train :  0.7682
Pire Résultat sur la val :  0.5387
Meilleur résultat sur le train :  0.97862
Meilleur résultat sur la val :  0.6023
Moyenne des résultats sur le train :  0.9164833333333334
Moyenne des résultats sur la val :  0.571975
Mediane des résultat sur le train :  0.9565299999999999
Mediane des résultat sur la val :  0.5767


### Les modèle dont la val est > à la moyenne

In [88]:
best_val_models

,nb_hidden_layers,filters,kernel_size,batch_size,input_shape,layers_activation,output_activation,use_skip,nb_skip,use_dropout,...,regulization_indexes,loss,optimizer,metrics,padding,epochs,model_id,train_accuracy,val_accuracy,dossier
3,6,64,3 3,64,32 32 3,relu,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,4288707,0.97262,0.6023,20200128
8,8,64,3 3,128,32 32 3,relu,softmax,True,2,True,...,4 6,categorical_crossentropy,Adam,categorical_accuracy,same,50,7460384,0.77644,0.5969,20200205
4,8,64,3 3,64,32 32 3,relu,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,2747992,0.97862,0.5944,20200127
3,8,64,3 3,64,32 32 3,selu,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,9276555,0.97346,0.5875,20200127
4,6,32,3 3,64,32 32 3,softmax,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,5239202,0.97038,0.5873,20200128
7,8,64,3 3,128,32 32 3,relu,softmax,True,2,True,...,4,categorical_crossentropy,Adam,categorical_accuracy,same,50,1345159,0.76820,0.5828,20200205
2,6,32,3 3,64,32 32 3,relu,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,5627949,0.96512,0.5706,20200128
2,6,64,3 3,64,32 32 3,softmax,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,360263,0.95868,0.5698,20200128
3,6,32,3 3,64,32 32 3,softmax,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,5333719,0.93950,0.5533,20200128
3,4,64,3 3,256,32 32 3,relu,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,30,9861705,0.86110,0.5403,20200126


###### On voit donc que nous avons 10 modèles qui pourrait être intéressants sur la validation, mais nous allons maintenant en selectionné, entre ceux là, les meilleurs sur le train. 

## Selection des meilleurs modèles sur la val sur les 10 restants:

In [90]:
best_val_models.to_csv("{}best_Rsnet_without_regularisation.csv".format(root_csv_folder), sep=';',header=True, index=False)

## Conclusions premiers tests:

##### Avec cette première vague de tests, nous avons vu que des Rsnet, avec différents nombres de couches cachées, ne dépasent pas les 60% sur la val .

##### Cependant, on a aussi remarqué que tous ces modèles on tendance à beacoup overfitter. Du coup on va essayer d'appliquer des techniques de régulatisation pour voir si on peu tirer de meilleurs résultats avec les meilleurs modèles qu'on a réussi à avoir.

##### Nous avons selectionnés deux modèles pour poursuivre les tests : le meilleur modèle qu'on a obtenu (6 couches avec un filtre de 64  avec une relu pour l'activation des couches cachées et une softmax pour l'output) et un modèle qui overfitte beaucoup  (6 couches avec un filtre de 64  avec une relu pour les couches cachées et la softmax pour l'output). 

# -------------------------------------------------------------------------------------------------------------

# Rsnet Avec régularisation

   Pour la suite de nos tests, nous avons lancer une nouvelle vague de tests avec régularisation, suite au constat de l'overfiting sur les tests précédents, sur différents Rsnet en jouant sur le nombre de couches ainsi que leur filtres. Pour cela, on a lancé des tests comme suit (plusieurs fois sur la même structure) :
    - Des modèles de 3 à 4 couches, avec filtres entre 32 et 64.
    - Des modèles de 5 à 6 couches, avec filtres entre 32 et 64.
    - Des modèles de 7 à 8 couches, avec filtres entre 32 et 64.
 
    
    Nous avons ensuite enregistré les résultats obtenus dans des fichiers CSV, en plus des logs générés par tensorflow, pour étudier les résultats obtenus.
    
    Nous allons donc charger ces statistiques et voir ce que ce genre de modèles peut donner sur le Dataset 

#### Dropout régularisation

In [91]:
results_dataframes_list = []
results_dataframes_desc = []
root_csv_folder = "..\\historique_tests\\"


# Rsnet with regularisation dropout results  3 to 4 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_dropout_3_to_4_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_dropout_3_to_4_{}".format(i))

#Rsnet with regularisation dropout results  5 to 6 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_dropout_5_to_6_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_dropout_5_to_6_{}".format(i))

# Rsnet with regularisation dropout results  7 to 8 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_dropout_7_to_8_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_dropout_7_to_8_{}".format(i))



In [92]:
for i in range(len(results_dataframes_list)):
    history = results_dataframes_list[i]
    history_desc = results_dataframes_desc[i]
    print_statistics(history, history_desc)


########### Models rsnet_dropout_3_to_4_32 ###########
(6, 26)
Pire résultat sur le train :  0.4397
Pire Résultat sur la val :  0.4036
Meilleur résultat sur le train :  0.85782
Meilleur résultat sur la val :  0.5846
Moyenne des résultats sur le train :  0.6423233333333332
Moyenne des résultats sur la val :  0.48100000000000004
Mediane des résultat sur le train :  0.63984
Mediane des résultat sur la val :  0.4798

########### Models rsnet_dropout_3_to_4_64 ###########
(8, 26)
Pire résultat sur le train :  0.29814
Pire Résultat sur la val :  0.2536
Meilleur résultat sur le train :  0.6977
Meilleur résultat sur la val :  0.5185
Moyenne des résultats sur le train :  0.5182775
Moyenne des résultats sur la val :  0.42691250000000003
Mediane des résultat sur le train :  0.50337
Mediane des résultat sur la val :  0.4334

########### Models rsnet_dropout_5_to_6_32 ###########
(3, 26)
Pire résultat sur le train :  0.65422
Pire Résultat sur la val :  0.5008
Meilleur résultat sur le train :  0.98

#### On constate toujours un overfitting même aprés un dropout, et aussi quand la valeur du dropout est trop inportante, la val et la train accuracy sont toutes deux basses.

###### On ne garde que les modèles dont les résultats sur la validations sont supérieurs à la moyenne des résultats de tous les modèles (d'une même architecture) testés 

In [93]:
filtered_results_dataframes = filter_on_mean_val_accuracy(results_dataframes_list)

## Selection des meilleurs modèles sur la val sur les modèles selectionnés:
    Pour cela, nous n'allons prendre que les modèles dont les résultats sur l'entraînement sont supérieurs à la moyenne des résultats de tous les modèles compris (toutes arachitectures confondues).

In [94]:
for i in range(len(filtered_results_dataframes)):
    filtered_history = filtered_results_dataframes[i]
    filtered_history_desc = results_dataframes_desc[i]
    print_statistics(filtered_history, filtered_history_desc)


########### Models rsnet_dropout_3_to_4_32 ###########
(2, 26)
Pire résultat sur le train :  0.5826399999999999
Pire Résultat sur la val :  0.5226
Meilleur résultat sur le train :  0.85782
Meilleur résultat sur la val :  0.5846
Moyenne des résultats sur le train :  0.7202299999999999
Moyenne des résultats sur la val :  0.5536
Mediane des résultat sur le train :  0.7202299999999999
Mediane des résultat sur la val :  0.5536

########### Models rsnet_dropout_3_to_4_64 ###########
(4, 26)
Pire résultat sur le train :  0.5492199999999999
Pire Résultat sur la val :  0.4492
Meilleur résultat sur le train :  0.6977
Meilleur résultat sur la val :  0.5185
Moyenne des résultats sur le train :  0.636185
Moyenne des résultats sur la val :  0.49215
Mediane des résultat sur le train :  0.64891
Mediane des résultat sur la val :  0.5004500000000001

########### Models rsnet_dropout_5_to_6_32 ###########
(1, 26)
Pire résultat sur le train :  0.98716
Pire Résultat sur la val :  0.6014
Meilleur résultat 

###### On fusionne tous les dataframes en une seule (étant donné qu'il y en a une par structure de modèle)

In [95]:
merged_df = concat_list_of_dataframes(filtered_results_dataframes)

In [96]:
print_statistics(merged_df, "Rsnet_with_dropout_regularisation")


########### Models Rsnet_with_dropout_regularisation ###########
(13, 26)
Pire résultat sur le train :  0.5492199999999999
Pire Résultat sur la val :  0.4492
Meilleur résultat sur le train :  0.98716
Meilleur résultat sur la val :  0.6014
Moyenne des résultats sur le train :  0.7728876923076923
Moyenne des résultats sur la val :  0.5305000000000001
Mediane des résultat sur le train :  0.6977
Mediane des résultat sur la val :  0.5226


In [97]:
best_val_models = filter_on_mean_val_accuracy([merged_df])[0]
best_val_models = best_val_models.sort_values('val_accuracy', ascending=False)
print_statistics(best_val_models, "Best Rsnet Dropout models")


########### Models Best Rsnet Dropout models ###########
(5, 26)
Pire résultat sur le train :  0.85782
Pire Résultat sur la val :  0.5601
Meilleur résultat sur le train :  0.98716
Meilleur résultat sur la val :  0.6014
Moyenne des résultats sur le train :  0.9387240000000002
Moyenne des résultats sur la val :  0.585
Mediane des résultat sur le train :  0.9605600000000001
Mediane des résultat sur la val :  0.5847


### Les modèle dont la val est > à la moyenne

In [98]:
best_val_models

,nb_hidden_layers,filters,kernel_size,batch_size,input_shape,layers_activation,output_activation,use_skip,nb_skip,use_dropout,...,regulization_indexes,loss,optimizer,metrics,padding,epochs,model_id,train_accuracy,val_accuracy,dossier
0,6,32,4 4,128,32 32 3,softmax,softmax,True,2,True,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,6251911,0.98716,0.6014,20200208
0,8,64,3 3,64,32 32 3,relu,softmax,True,2,True,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,2201654,0.97288,0.5942,20200128
3,6,64,4 4,128,32 32 3,selu,softmax,True,2,True,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,4949333,0.96056,0.5847,20200208
0,3,64,3 3,256,32 32 3,softmax,softmax,True,2,True,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,30,6932724,0.85782,0.5846,20200125
0,6,64,3 3,128,32 32 3,softmax,softmax,True,2,True,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,8863967,0.91520,0.5601,20200208


In [99]:
best_val_models.to_csv("{}best_Rsnet_with_dropout_regularisation.csv".format(root_csv_folder), sep=';',header=True, index=False)

### Exemple Rnset Dropout:

![Rsnet Dropout](Rsnet_Dropout.png)

#### Maintenant nous allons essayer La L1L2 régularisation 

#### L1L2 régularisation


In [100]:
results_dataframes_list = []
results_dataframes_desc = []
root_csv_folder = "..\\historique_tests\\"


# Rsnet with regularisation L1L2 results  3 to 4 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_regul_3_to_4_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_regul_3_to_4_{}".format(i))

#Rsnet with regularisation L1L2 results  5 to 6 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_regul_5_to_6_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_regul_5_to_6_{}".format(i))

# Rsnet with regularisation L1L2 results  7 to 8 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_regul_7_to_8_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_regul_7_to_8_{}".format(i))



In [101]:
for i in range(len(results_dataframes_list)):
    history = results_dataframes_list[i]
    history_desc = results_dataframes_desc[i]
    print_statistics(history, history_desc)


########### Models rsnet_regul_3_to_4_32 ###########
(8, 26)
Pire résultat sur le train :  0.1
Pire Résultat sur la val :  0.1
Meilleur résultat sur le train :  0.4411
Meilleur résultat sur la val :  0.4357
Moyenne des résultats sur le train :  0.324335
Moyenne des résultats sur la val :  0.3206625
Mediane des résultat sur le train :  0.34986999999999996
Mediane des résultat sur la val :  0.34295

########### Models rsnet_regul_3_to_4_64 ###########
(7, 26)
Pire résultat sur le train :  0.12868
Pire Résultat sur la val :  0.1266
Meilleur résultat sur le train :  0.36582
Meilleur résultat sur la val :  0.3588
Moyenne des résultats sur le train :  0.28935714285714287
Moyenne des résultats sur la val :  0.28452857142857146
Mediane des résultat sur le train :  0.35266
Mediane des résultat sur la val :  0.3446

########### Models rsnet_regul_5_to_6_32 ###########
(5, 26)
Pire résultat sur le train :  0.26542
Pire Résultat sur la val :  0.2682
Meilleur résultat sur le train :  0.6153
Meille

#### Ici on ne constate le cas d'overfiting et le % de la val et du train est très bas, car la valeur de la L1L2 est trop importante. Car au depart on a lancé des tests random du coup les valeurs de la L1L2 retournées ont fais que les modèles n'arrivent pas à dépasser l'intervale 30 à 40 %. En suite on a ajuster les valeurs de la L1L2 (moins elevées) pour avoir de mielleurs résultats d'accuracy.



###### On ne garde que les modèles dont les résultats sur la validations sont supérieurs à la moyenne des résultats de tous les modèles (d'une même architecture) testés 

In [102]:
filtered_results_dataframes = filter_on_mean_val_accuracy(results_dataframes_list)

## Selection des meilleurs modèles sur la val sur les modèles selectionnés:
    Pour cela, nous n'allons prendre que les modèles dont les résultats sur l'entraînement sont supérieurs à la moyenne des résultats de tous les modèles compris (toutes arachitectures confondues).

In [103]:
for i in range(len(filtered_results_dataframes)):
    filtered_history = filtered_results_dataframes[i]
    filtered_history_desc = results_dataframes_desc[i]
    print_statistics(filtered_history, filtered_history_desc)


########### Models rsnet_regul_3_to_4_32 ###########
(5, 26)
Pire résultat sur le train :  0.34431999999999996
Pire Résultat sur la val :  0.3361
Meilleur résultat sur le train :  0.4411
Meilleur résultat sur la val :  0.4357
Moyenne des résultats sur le train :  0.38769200000000004
Moyenne des résultats sur la val :  0.38306
Mediane des résultat sur le train :  0.36404000000000003
Mediane des résultat sur la val :  0.3649

########### Models rsnet_regul_3_to_4_64 ###########
(5, 26)
Pire résultat sur le train :  0.28665999999999997
Pire Résultat sur la val :  0.2912
Meilleur résultat sur le train :  0.36582
Meilleur résultat sur la val :  0.3588
Moyenne des résultats sur le train :  0.346748
Moyenne des résultats sur la val :  0.34030000000000005
Mediane des résultat sur le train :  0.36401999999999995
Mediane des résultat sur la val :  0.3533

########### Models rsnet_regul_5_to_6_32 ###########
(2, 26)
Pire résultat sur le train :  0.60044
Pire Résultat sur la val :  0.5635
Meilleu

###### On fusionne tous les dataframes en une seule (étant donné qu'il y en a une par structure de modèle)

In [104]:
merged_df = concat_list_of_dataframes(filtered_results_dataframes)

In [105]:
print_statistics(merged_df, "Rsnet_with_L1L2_regularisation")


########### Models Rsnet_with_L1L2_regularisation ###########
(19, 26)
Pire résultat sur le train :  0.272
Pire Résultat sur la val :  0.2713
Meilleur résultat sur le train :  0.67912
Meilleur résultat sur la val :  0.6259
Moyenne des résultats sur le train :  0.40738526315789475
Moyenne des résultats sur la val :  0.39569473684210527
Mediane des résultat sur le train :  0.36582
Mediane des résultat sur la val :  0.3588


In [106]:
best_val_models = filter_on_mean_val_accuracy([merged_df])[0]
best_val_models = best_val_models.sort_values('val_accuracy', ascending=False)
print_statistics(best_val_models, "Best Rsnet L1L2 models")


########### Models Best Rsnet L1L2 models ###########
(6, 26)
Pire résultat sur le train :  0.43358
Pire Résultat sur la val :  0.4288
Meilleur résultat sur le train :  0.67912
Meilleur résultat sur la val :  0.6259
Moyenne des résultats sur le train :  0.5425833333333333
Moyenne des résultats sur la val :  0.5190666666666667
Mediane des résultat sur le train :  0.5432
Mediane des résultat sur la val :  0.5182


In [107]:
best_val_models

,nb_hidden_layers,filters,kernel_size,batch_size,input_shape,layers_activation,output_activation,use_skip,nb_skip,use_dropout,...,regulization_indexes,loss,optimizer,metrics,padding,epochs,model_id,train_accuracy,val_accuracy,dossier
3,8,32,3 3,128,32 32 3,relu,softmax,True,2,False,...,5,categorical_crossentropy,Adam,categorical_accuracy,same,50,1477028,0.67912,0.6259,20200209
3,5,32,3 3,128,32 32 3,selu,softmax,True,2,False,...,3,categorical_crossentropy,Adam,categorical_accuracy,same,50,3162669,0.61530,0.5876,20200209
4,6,32,3 3,128,32 32 3,relu,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,4359082,0.60044,0.5635,20200209
5,7,32,3 3,128,32 32 3,softmax,softmax,True,2,False,...,6 5,categorical_crossentropy,Adam,categorical_accuracy,same,50,7466498,0.48596,0.4729,20200209
7,4,32,3 3,128,32 32 3,softmax,softmax,True,2,False,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,3333723,0.43358,0.4357,20200209
6,3,32,3 3,128,32 32 3,relu,softmax,True,2,False,...,2,categorical_crossentropy,Adam,categorical_accuracy,same,50,3787713,0.44110,0.4288,20200209


In [108]:
best_val_models.to_csv("{}best_Rsnet_with_L1L2_regularisation.csv".format(root_csv_folder), sep=';',header=True, index=False)

### Exemple de progression du loss et de l'accuracy avec L1L2 regularisation:

![tensorbord L1L2](Rsnet_Regul.PNG)

### Exemple d'un Rsnet avec L1L2 regularisation:

![Rsnet L1L2](Rsnet_regul.png)

#### Maintenant nous allons combiner les deux techniques de regularisations; L1L2 et Dropout:

#### L1L2 and Dropout régularisation

In [109]:
results_dataframes_list = []
results_dataframes_desc = []
root_csv_folder = "..\\historique_tests\\"


# Rsnet with regularisation L1L2 and dropout results  3 to 4 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_rg_dr_3_to_4_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_rg_dr_3_to_4_{}".format(i))

#Rsnet with regularisation L1L2 and dropout results  5 to 6 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_rg_dr_5_to_6_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_rg_dr_5_to_6_{}".format(i))

# Rsnet with regularisation L1L2 and dropout results  7 to 8 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_rg_dr_7_to_8_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_rg_dr_7_to_8_{}".format(i))



In [110]:
for i in range(len(results_dataframes_list)):
    history = results_dataframes_list[i]
    history_desc = results_dataframes_desc[i]
    print_statistics(history, history_desc)


########### Models rsnet_rg_dr_3_to_4_32 ###########
(6, 26)
Pire résultat sur le train :  0.11606
Pire Résultat sur la val :  0.114
Meilleur résultat sur le train :  0.4722
Meilleur résultat sur la val :  0.4629
Moyenne des résultats sur le train :  0.25787
Moyenne des résultats sur la val :  0.2558666666666667
Mediane des résultat sur le train :  0.23465
Mediane des résultat sur la val :  0.2353

########### Models rsnet_rg_dr_3_to_4_64 ###########
(6, 26)
Pire résultat sur le train :  0.11027999999999999
Pire Résultat sur la val :  0.1111
Meilleur résultat sur le train :  0.43648000000000003
Meilleur résultat sur la val :  0.4241
Moyenne des résultats sur le train :  0.2891933333333333
Moyenne des résultats sur la val :  0.2855
Mediane des résultat sur le train :  0.30627
Mediane des résultat sur la val :  0.30265

########### Models rsnet_rg_dr_5_to_6_32 ###########
(3, 26)
Pire résultat sur le train :  0.1333
Pire Résultat sur la val :  0.1314
Meilleur résultat sur le train :  0.

#### On constate qu'avec les deux methodes de regularisation ensemles que la val et le train ne sont pas très grand, mais on constate que l'overfiting est moins imporant ( mise à part quelques cas ) car le jeux de tests lancé est fait avec des random sur les valeurs de regularisation et les couches ciblees, puis on ajuste les valeurs.

###### On ne garde que les modèles dont les résultats sur la validations sont supérieurs à la moyenne des résultats de tous les modèles (d'une même architecture) testés 

In [111]:
filtered_results_dataframes = filter_on_mean_val_accuracy(results_dataframes_list)

## Selection des meilleurs modèles sur la val sur les modèles selectionnés:
    Pour cela, nous n'allons prendre que les modèles dont les résultats sur l'entraînement sont supérieurs à la moyenne des résultats de tous les modèles compris (toutes arachitectures confondues).

In [112]:
for i in range(len(filtered_results_dataframes)):
    filtered_history = filtered_results_dataframes[i]
    filtered_history_desc = results_dataframes_desc[i]
    print_statistics(filtered_history, filtered_history_desc)


########### Models rsnet_rg_dr_3_to_4_32 ###########
(2, 26)
Pire résultat sur le train :  0.30201999999999996
Pire Résultat sur la val :  0.2998
Meilleur résultat sur le train :  0.4722
Meilleur résultat sur la val :  0.4629
Moyenne des résultats sur le train :  0.38710999999999995
Moyenne des résultats sur la val :  0.38134999999999997
Mediane des résultat sur le train :  0.38710999999999995
Mediane des résultat sur la val :  0.38134999999999997

########### Models rsnet_rg_dr_3_to_4_64 ###########
(4, 26)
Pire résultat sur le train :  0.29334
Pire Résultat sur la val :  0.2922
Meilleur résultat sur le train :  0.43648000000000003
Meilleur résultat sur la val :  0.4241
Moyenne des résultats sur le train :  0.362135
Moyenne des résultats sur la val :  0.355575
Mediane des résultat sur le train :  0.35936
Mediane des résultat sur la val :  0.353

########### Models rsnet_rg_dr_5_to_6_32 ###########
(1, 26)
Pire résultat sur le train :  0.21975999999999998
Pire Résultat sur la val :  0

###### On fusionne tous les dataframes en une seule (étant donné qu'il y en a une par structure de modèle)

In [113]:
merged_df = concat_list_of_dataframes(filtered_results_dataframes)

In [114]:
print_statistics(merged_df, "Rsnet_with_droupout_L1L2_regularisation")


########### Models Rsnet_with_droupout_L1L2_regularisation ###########
(24, 26)
Pire résultat sur le train :  0.21975999999999998
Pire Résultat sur la val :  0.2246
Meilleur résultat sur le train :  0.9707600000000001
Meilleur résultat sur la val :  0.6512
Moyenne des résultats sur le train :  0.6317875000000001
Moyenne des résultats sur la val :  0.5107541666666667
Mediane des résultat sur le train :  0.71773
Mediane des résultat sur la val :  0.5559000000000001


In [115]:
best_val_models = filter_on_mean_val_accuracy([merged_df])[0]
best_val_models = best_val_models.sort_values('val_accuracy', ascending=False)
print_statistics(best_val_models, "Best Rsnet Dropout L1L2 models")


########### Models Best Rsnet Dropout L1L2 models ###########
(15, 26)
Pire résultat sur le train :  0.54138
Pire Résultat sur la val :  0.5274
Meilleur résultat sur le train :  0.9707600000000001
Meilleur résultat sur la val :  0.6512
Moyenne des résultats sur le train :  0.762348
Moyenne des résultats sur la val :  0.5899666666666668
Mediane des résultat sur le train :  0.75864
Mediane des résultat sur la val :  0.5894


In [116]:
best_val_models

,nb_hidden_layers,filters,kernel_size,batch_size,input_shape,layers_activation,output_activation,use_skip,nb_skip,use_dropout,...,regulization_indexes,loss,optimizer,metrics,padding,epochs,model_id,train_accuracy,val_accuracy,dossier
0,8,64,3 3,128,32 32 3,relu,softmax,True,2,True,...,2 4,categorical_crossentropy,Adam,categorical_accuracy,same,50,45286,0.87504,0.6512,20200209
13,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,2,categorical_crossentropy,Adam,categorical_accuracy,same,50,2599981,0.76114,0.6438,20200203
11,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,2,categorical_crossentropy,Adam,categorical_accuracy,same,50,7171554,0.77956,0.6382,20200203
22,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,3 6,categorical_crossentropy,Adam,categorical_accuracy,same,50,4893841,0.72964,0.6380,20200202
14,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,2,categorical_crossentropy,Adam,categorical_accuracy,same,50,2775255,0.73240,0.6187,20200203
20,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,3 6,categorical_crossentropy,Adam,categorical_accuracy,same,50,5325785,0.71808,0.5922,20200202
5,6,64,3 3,128,32 32 3,selu,softmax,True,2,True,...,2 5,categorical_crossentropy,Adam,categorical_accuracy,same,50,5165771,0.97076,0.5905,20200203
21,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,3 6,categorical_crossentropy,Adam,categorical_accuracy,same,50,2349585,0.71738,0.5894,20200202
15,5,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,4,categorical_crossentropy,Adam,categorical_accuracy,same,50,4113825,0.78162,0.5742,20200204
12,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,2,categorical_crossentropy,Adam,categorical_accuracy,same,50,8176707,0.72282,0.5711,20200203


In [117]:
best_val_models.to_csv("{}best_Rsnet_with_Dropout_L1L2_regularisation.csv".format(root_csv_folder), sep=';',header=True, index=False)

### On constate que la régularisation à bien amoindri l'overfiting, on constate que l'ecart entre la val et le train et de plus en plus petit.

### Tensorbord Rsnet Dropout et L1L2:


![Dropout_L1L2](Dropout_L1L2.PNG)

###  Exemple Rsnet Dropout et L1L2 :

![Rsnet Dropout L1L2](Rsnet_Dropout_L1L2.png)

### Après avoir fait divers tests, on a constaté que la val n'arrivait pas à depasser les 65% environs et cela malgré les diverses techniques utilisées. Il nous restait à tester les Rsnet avec les Maxpooling combiné avec Dropout et L1L2 regularisation ( mission jugée un peut délicate au début en vue de l'utilisation des add() donc problèmes de shapes, mais ça a pu se faire en utilisant un UpSampling2D avant le add() ) afin de voir si on pouvais amméliorer la val accuracy.

#### Use Maxpoolling

In [118]:
results_dataframes_list = []
results_dataframes_desc = []
root_csv_folder = "..\\historique_tests\\"


# Rsnet with Maxpooling results 5 to 6 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_maxpool_5_to_6_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_maxpool_5_to_6_{}".format(i))

#Rsnet with Maxpooling results  7 to 8 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_maxpool_7_to_8_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_maxpool_7_to_8_{}".format(i))

# Rsnet with Maxpooling results  9 layers.
for i in [32,64]:
    df = pd.read_csv("{}tested_rsnet_maxpool_9_{}_history.csv".format(root_csv_folder,i), sep=";")
    results_dataframes_list.append(df)
    results_dataframes_desc.append("rsnet_maxpool_9_{}".format(i))

In [119]:
# faut lancer des tests sur maxpool 5 to 6 64
for i in range(len(results_dataframes_list)):
    history = results_dataframes_list[i]
    history_desc = results_dataframes_desc[i]
    print_statistics(history, history_desc)


########### Models rsnet_maxpool_5_to_6_32 ###########
(6, 26)
Pire résultat sur le train :  0.45364
Pire Résultat sur la val :  0.4476
Meilleur résultat sur le train :  0.8593799999999999
Meilleur résultat sur la val :  0.6222
Moyenne des résultats sur le train :  0.6746833333333333
Moyenne des résultats sur la val :  0.5539833333333333
Mediane des résultat sur le train :  0.69415
Mediane des résultat sur la val :  0.57315

########### Models rsnet_maxpool_5_to_6_64 ###########
(3, 26)
Pire résultat sur le train :  0.41648
Pire Résultat sur la val :  0.3992
Meilleur résultat sur le train :  0.8658600000000001
Meilleur résultat sur la val :  0.6229
Moyenne des résultats sur le train :  0.6055133333333333
Moyenne des résultats sur la val :  0.5109
Mediane des résultat sur le train :  0.5342
Mediane des résultat sur la val :  0.5106

########### Models rsnet_maxpool_7_to_8_32 ###########
(4, 26)
Pire résultat sur le train :  0.49928
Pire Résultat sur la val :  0.4856
Meilleur résultat s

###### On ne garde que les modèles dont les résultats sur la validations sont supérieurs à la moyenne des résultats de tous les modèles (d'une même architecture) testés 

In [120]:
filtered_results_dataframes = filter_on_mean_val_accuracy(results_dataframes_list)

## Selection des meilleurs modèles sur la val sur les modèles selectionnés:
    Pour cela, nous n'allons prendre que les modèles dont les résultats sur l'entraînement sont supérieurs à la moyenne des résultats de tous les modèles compris (toutes arachitectures confondues).

In [121]:
for i in range(len(filtered_results_dataframes)):
    filtered_history = filtered_results_dataframes[i]
    filtered_history_desc = results_dataframes_desc[i]
    print_statistics(filtered_history, filtered_history_desc)


########### Models rsnet_maxpool_5_to_6_32 ###########
(4, 26)
Pire résultat sur le train :  0.5791
Pire Résultat sur la val :  0.5602
Meilleur résultat sur le train :  0.8593799999999999
Meilleur résultat sur la val :  0.6222
Moyenne des résultats sur le train :  0.7702
Moyenne des résultats sur la val :  0.597375
Mediane des résultat sur le train :  0.8211600000000001
Mediane des résultat sur la val :  0.60355

########### Models rsnet_maxpool_5_to_6_64 ###########
(1, 26)
Pire résultat sur le train :  0.8658600000000001
Pire Résultat sur la val :  0.6229
Meilleur résultat sur le train :  0.8658600000000001
Meilleur résultat sur la val :  0.6229
Moyenne des résultats sur le train :  0.8658600000000001
Moyenne des résultats sur la val :  0.6229
Mediane des résultat sur le train :  0.8658600000000001
Mediane des résultat sur la val :  0.6229

########### Models rsnet_maxpool_7_to_8_32 ###########
(3, 26)
Pire résultat sur le train :  0.6956
Pire Résultat sur la val :  0.6314
Meilleur 

###### On fusionne tous les dataframes en une seule (étant donné qu'il y en a une par structure de modèle)

In [122]:
merged_df = concat_list_of_dataframes(filtered_results_dataframes)

In [123]:
print_statistics(merged_df, "Rsnet_with_maxpool_regularisation")


########### Models Rsnet_with_maxpool_regularisation ###########
(16, 26)
Pire résultat sur le train :  0.5791
Pire Résultat sur la val :  0.5602
Meilleur résultat sur le train :  0.8873200000000001
Meilleur résultat sur la val :  0.7121
Moyenne des résultats sur le train :  0.77987875
Moyenne des résultats sur la val :  0.63400625
Mediane des résultat sur le train :  0.8160700000000001
Mediane des résultat sur la val :  0.62715


In [124]:
best_val_models = filter_on_mean_val_accuracy([merged_df])[0]
best_val_models = best_val_models.sort_values('val_accuracy', ascending=False)
print_statistics(best_val_models, "Best Rsnet Maxpool Regularisation models")


########### Models Best Rsnet Maxpool Regularisation models ###########
(7, 26)
Pire résultat sur le train :  0.74526
Pire Résultat sur la val :  0.6358
Meilleur résultat sur le train :  0.8873200000000001
Meilleur résultat sur la val :  0.7121
Moyenne des résultats sur le train :  0.8171285714285715
Moyenne des résultats sur la val :  0.6685857142857142
Mediane des résultat sur le train :  0.8221
Mediane des résultat sur la val :  0.6686


In [125]:
best_val_models

,nb_hidden_layers,filters,kernel_size,batch_size,input_shape,layers_activation,output_activation,use_skip,nb_skip,use_dropout,...,regulization_indexes,loss,optimizer,metrics,padding,epochs,model_id,train_accuracy,val_accuracy,dossier
6,7,64,3 3,128,32 32 3,selu,softmax,True,2,True,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,9851172,0.82210,0.7121,20200209
2,7,64,3 3,128,32 32 3,selu,softmax,True,2,True,...,2 2 7,categorical_crossentropy,Adam,categorical_accuracy,same,50,1250532,0.78266,0.6965,20200208
3,7,32,3 3,128,32 32 3,relu,softmax,True,2,True,...,2 4,categorical_crossentropy,Adam,categorical_accuracy,same,50,5097248,0.82240,0.6757,20200209
2,7,32,3 3,128,32 32 3,selu,softmax,True,2,True,...,2 4,categorical_crossentropy,Adam,categorical_accuracy,same,50,7628417,0.88732,0.6686,20200209
5,7,64,3 3,128,32 32 3,selu,softmax,True,2,True,...,2 7,categorical_crossentropy,Adam,categorical_accuracy,same,50,638853,0.74526,0.6511,20200209
3,7,64,3 3,128,32 32 3,relu,softmax,True,2,True,...,4 6,categorical_crossentropy,Adam,categorical_accuracy,same,50,4666378,0.81004,0.6403,20200206
4,7,64,3 3,128,32 32 3,relu,softmax,True,2,True,...,4 6,categorical_crossentropy,Adam,categorical_accuracy,same,50,2279421,0.85012,0.6358,20200206


In [126]:
best_val_models.to_csv("{}best_Rsnet_with_Maxpool_regularisation.csv".format(root_csv_folder), sep=';',header=True, index=False)

### Exemple  d'un Rsnet avec du Maxpooling:

![Rsnet Maxpool](Rsnet_Maxpool.png)

#### L'image ci-dessous décrit l'evolution d'un Rsnet avec Maxpooling

![Rsnet MaxPool Log](Rsnet_Maxpool_log.PNG)

### Effectivement avec le Maxpooling combiné avec de la regularisation (L1L2 et Dropout)  on est arrivé à de meilleurs résultats, soit une val accuracy près de 70%.

## Maintenant on vas recupperer tous nos bests models (avec / sans regularisation, dropout, les deux ou maxpool), pour en sortir les meilleurs des meilleurs !

In [127]:
results_dataframes_list = []
results_dataframes_desc = []
root_csv_folder = "..\\historique_tests\\"


# Rsnet best  without regularisation .
df = pd.read_csv("{}best_Rsnet_without_regularisation.csv".format(root_csv_folder), sep=";")
results_dataframes_list.append(df)
results_dataframes_desc.append("best_Rsnet_without_regularisation")

#Rsnet best with dropout regularisation.
df = pd.read_csv("{}best_Rsnet_with_dropout_regularisation.csv".format(root_csv_folder), sep=";")
results_dataframes_list.append(df)
results_dataframes_desc.append("best_Rsnet_with_dropout_regularisation")

#Rsnet best with L1L2 regularisation.
df = pd.read_csv("{}best_Rsnet_with_L1L2_regularisation.csv".format(root_csv_folder), sep=";")
results_dataframes_list.append(df)
results_dataframes_desc.append("best_Rsnet_with_L1L2_regularisation")

#Rsnet best with Dropout and L1L2 regularisation.
df = pd.read_csv("{}best_Rsnet_with_Dropout_L1L2_regularisation.csv".format(root_csv_folder), sep=";")
results_dataframes_list.append(df)
results_dataframes_desc.append("best_Rsnet_with_Dropout_L1L2_regularisation")

#Rsnet best with Maxpooling regularisation.
df = pd.read_csv("{}best_Rsnet_with_Maxpool_regularisation.csv".format(root_csv_folder), sep=";")
results_dataframes_list.append(df)
results_dataframes_desc.append("best_Rsnet_with_Maxpool_regularisation")

In [128]:
for i in range(len(results_dataframes_list)):
    history = results_dataframes_list[i]
    history_desc = results_dataframes_desc[i]
    print_statistics(history, history_desc)


########### Models best_Rsnet_without_regularisation ###########
(12, 26)
Pire résultat sur le train :  0.7682
Pire Résultat sur la val :  0.5387
Meilleur résultat sur le train :  0.97862
Meilleur résultat sur la val :  0.6023
Moyenne des résultats sur le train :  0.9164833333333334
Moyenne des résultats sur la val :  0.571975
Mediane des résultat sur le train :  0.95653
Mediane des résultat sur la val :  0.5767

########### Models best_Rsnet_with_dropout_regularisation ###########
(5, 26)
Pire résultat sur le train :  0.85782
Pire Résultat sur la val :  0.5601
Meilleur résultat sur le train :  0.98716
Meilleur résultat sur la val :  0.6014
Moyenne des résultats sur le train :  0.9387239999999999
Moyenne des résultats sur la val :  0.585
Mediane des résultat sur le train :  0.96056
Mediane des résultat sur la val :  0.5847

########### Models best_Rsnet_with_L1L2_regularisation ###########
(6, 26)
Pire résultat sur le train :  0.43358
Pire Résultat sur la val :  0.4288
Meilleur résult

###### On ne garde que les modèles dont les résultats sur la validations sont supérieurs à la moyenne des résultats de tous les modèles (d'une même architecture) testés 

In [129]:
filtered_results_dataframes = filter_on_mean_val_accuracy(results_dataframes_list)

In [130]:
for i in range(len(filtered_results_dataframes)):
    filtered_history = filtered_results_dataframes[i]
    filtered_history_desc = results_dataframes_desc[i]
    print_statistics(filtered_history, filtered_history_desc)


########### Models best_Rsnet_without_regularisation ###########
(6, 26)
Pire résultat sur le train :  0.7682
Pire Résultat sur la val :  0.5828
Meilleur résultat sur le train :  0.97862
Meilleur résultat sur la val :  0.6023
Moyenne des résultats sur le train :  0.9066200000000002
Moyenne des résultats sur la val :  0.5918666666666667
Mediane des résultat sur le train :  0.9715
Mediane des résultat sur la val :  0.5909500000000001

########### Models best_Rsnet_with_dropout_regularisation ###########
(2, 26)
Pire résultat sur le train :  0.97288
Pire Résultat sur la val :  0.5942
Meilleur résultat sur le train :  0.98716
Meilleur résultat sur la val :  0.6014
Moyenne des résultats sur le train :  0.98002
Moyenne des résultats sur la val :  0.5978
Mediane des résultat sur le train :  0.98002
Mediane des résultat sur la val :  0.5978

########### Models best_Rsnet_with_L1L2_regularisation ###########
(3, 26)
Pire résultat sur le train :  0.60044
Pire Résultat sur la val :  0.5635
Meill

###### On fusionne tous les dataframes en une seule (étant donné qu'il y en a une par structure de modèle)

In [131]:
merged_df = concat_list_of_dataframes(filtered_results_dataframes)

In [132]:
print_statistics(merged_df, "Best_Rsnet")


########### Models Best_Rsnet ###########
(22, 26)
Pire résultat sur le train :  0.60044
Pire Résultat sur la val :  0.5635
Meilleur résultat sur le train :  0.98716
Meilleur résultat sur la val :  0.7121
Moyenne des résultats sur le train :  0.826169090909091
Moyenne des résultats sur la val :  0.6204227272727274
Mediane des résultat sur le train :  0.8023800000000001
Mediane des résultat sur la val :  0.60185


In [133]:
best_val_models = filter_on_mean_val_accuracy([merged_df])[0]
best_val_models = best_val_models.sort_values('val_accuracy', ascending=False)
print_statistics(best_val_models, "Best Rsnet Maxpool Regularisation models")


########### Models Best Rsnet Maxpool Regularisation models ###########
(9, 26)
Pire résultat sur le train :  0.67912
Pire Résultat sur la val :  0.6259
Meilleur résultat sur le train :  0.8873200000000001
Meilleur résultat sur la val :  0.7121
Moyenne des résultats sur le train :  0.7932200000000001
Moyenne des résultats sur la val :  0.661111111111111
Mediane des résultat sur le train :  0.78266
Mediane des résultat sur la val :  0.6512


In [134]:
best_val_models

,nb_hidden_layers,filters,kernel_size,batch_size,input_shape,layers_activation,output_activation,use_skip,nb_skip,use_dropout,...,regulization_indexes,loss,optimizer,metrics,padding,epochs,model_id,train_accuracy,val_accuracy,dossier
0,7,64,3 3,128,32 32 3,selu,softmax,True,2,True,...,NaN,categorical_crossentropy,Adam,categorical_accuracy,same,50,9851172,0.82210,0.7121,20200209
1,7,64,3 3,128,32 32 3,selu,softmax,True,2,True,...,2 2 7,categorical_crossentropy,Adam,categorical_accuracy,same,50,1250532,0.78266,0.6965,20200208
2,7,32,3 3,128,32 32 3,relu,softmax,True,2,True,...,2 4,categorical_crossentropy,Adam,categorical_accuracy,same,50,5097248,0.82240,0.6757,20200209
3,7,32,3 3,128,32 32 3,selu,softmax,True,2,True,...,2 4,categorical_crossentropy,Adam,categorical_accuracy,same,50,7628417,0.88732,0.6686,20200209
0,8,64,3 3,128,32 32 3,relu,softmax,True,2,True,...,2 4,categorical_crossentropy,Adam,categorical_accuracy,same,50,45286,0.87504,0.6512,20200209
1,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,2,categorical_crossentropy,Adam,categorical_accuracy,same,50,2599981,0.76114,0.6438,20200203
2,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,2,categorical_crossentropy,Adam,categorical_accuracy,same,50,7171554,0.77956,0.6382,20200203
3,6,128,3 3,128,32 32 3,relu,softmax,True,2,True,...,3 6,categorical_crossentropy,Adam,categorical_accuracy,same,50,4893841,0.72964,0.6380,20200202
0,8,32,3 3,128,32 32 3,relu,softmax,True,2,False,...,5,categorical_crossentropy,Adam,categorical_accuracy,same,50,1477028,0.67912,0.6259,20200209


In [135]:
best_val_models.to_csv("{}best_Rsnet_last.csv".format(root_csv_folder), sep=';',header=True, index=False)

# Ci-dessous notre meilleurs modèle obtenu  Est le premier modèle dans le classement ci-dessus.

# Conclusion RSNET:

### Résumé de résultats obtenus:

#### Sur les premiers tests réalisés, on a obtenue un max val de 60% mais avec beaucoup d'overfiting, ensuite grâce aux techniques de régularisation on a pu résoudre tant bien que mal le problème de l'overfiting (bien entendu en ajustant nos tests après chaque resultats obtenus) c'est ainsi qu'on est arrivé vers les 64% avec un overfiting moins important. Puis, en vue du non évolutivité de la val accuracy, on a testé nos modèles avec du MaxPooling combiné avec de la L1L2 et du Dropout. Chose qui s'est avéré très fructueuse en vue de l'augmentation de la val acccuray qui avoisine maintenant les 71%.
